In [1]:
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import brier_score_loss, accuracy_score
import numpy as np
from scipy.optimize import minimize
from skqreg.estimators import QuantisticRegression

from sklearn.datasets import fetch_openml
from bisect import bisect_left
import matplotlib.pyplot as plt

In [2]:
X, y = fetch_openml(data_id=43979, return_X_y=True, as_frame=False)
y = np.array(y == "True", dtype=int)
y

/home/sheid/.cache/pypoetry/virtualenvs/scikit-quantistic-regresion-tuRLV2Fq-py3.11/lib/python3.11/site-packages/sklearn/datasets/_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


array([0, 0, 0, ..., 1, 1, 1])

In [8]:
clf = LogisticRegression(max_iter=500).fit(X,y)
clf.score(X,y)

0.8071629349617137

In [4]:
def f(s):
    c = clf.coef_
    c = c/np.abs(c).sum()*10*s
    return np.abs(np.abs(c.round()).sum() - 10)

x = minimize(f, x0=1, method="Nelder-Mead", options=dict(initial_simplex=[[.8],[1.2]], fatol=.9)).x.item()

c = clf.coef_
c = c/np.abs(c).sum()*10*x
n_cuts = c.round().astype(int).squeeze().tolist()
n_cuts

[3, 0, -1, 5, 0, -1, 0, 0]

In [5]:
iso = IsotonicRegression().fit(X[:,3],y)
iso.X_thresholds_,iso.y_thresholds_,np.linspace(0,1,num=5+2)[1:-1]

(array([ 0.33333333,  0.7012987 ,  0.70588235,  0.82191781,  0.82222222,
         0.875     ,  0.87573964,  0.92971246,  0.92977528,  0.92978723,
        34.06666667]),
 array([0.33333333, 0.33333333, 0.36231884, 0.36231884, 0.46456693,
        0.46456693, 0.49484536, 0.49484536, 0.5       , 0.50108438,
        0.50108438]),
 array([0.16666667, 0.33333333, 0.5       , 0.66666667, 0.83333333]))

In [6]:
cuts = []
for i,c in enumerate(n_cuts):
    if c == 0:
        continue
    iso = IsotonicRegression().fit(X[:,i],y)
    for split in np.linspace(0,1,num=abs(c)+2)[1:-1]:
        cuts.append([i,1 if c >=0 else -1, iso.X_thresholds_[min(bisect_left(iso.y_thresholds_, split),len(iso.X_thresholds_)-1)]] )
cuts

[[0, 1, 2.5658],
 [0, 1, 3.8365],
 [0, 1, 5.0016],
 [2, -1, 5.853300733496333],
 [3, 1, 0.3333333333333333],
 [3, 1, 0.3333333333333333],
 [3, 1, 0.9297752808988764],
 [3, 1, 34.06666666666667],
 [3, 1, 34.06666666666667],
 [5, -1, 1.3186003683241252]]

In [10]:
results = []
for f, sgn, cut in cuts:
    results.append(X[:,f] * sgn >= cut)
results = np.array(results, dtype=int)
ypred = results.sum(axis=0)/10
brier_score_loss(y,ypred), accuracy_score(y,ypred>.5)

(0.2053794707763885, 0.6777648541242609)